In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigs
import scipy.sparse as sp
from numba import njit

from randcolorgraphs.utils.calculate_katz import calculate_katz

@njit
def generate_edge_indices(n, p, seed):
    np.random.seed(seed)
    row_indices = []
    col_indices = []

    # Expected number of edges
    E = int(n * (n * p))

    # Generate E unique edge indices
    edge_set = set()
    while len(edge_set) < E:
        i = np.random.randint(0, n)
        j = np.random.randint(0, n)
        edge_set.add((i, j))

    # Separate the set into row and column indices
    for edge in edge_set:
        row_indices.append(edge[0])
        col_indices.append(edge[1])

    return row_indices, col_indices

def get_edge_vector_from_adj_matrix(adj_matrix):
    sparse_matrix = sp.csr_matrix(adj_matrix)
    row_indices, col_indices = sparse_matrix.nonzero()
    edge_vector = np.vstack((row_indices, col_indices)).T
    return edge_vector

def generate_edge_list_and_katz_vector(n, p, seed=42, alpha=0.1, beta=1.0):
    row_indices, col_indices = generate_edge_indices(n, p, seed)
    print("Got Edge Indices")

    data = np.ones(len(row_indices))
    adj_matrix = sp.csr_matrix((data, (row_indices, col_indices)), shape=(n, n), dtype=np.float64)

    eigenvalues, _ = eigs(adj_matrix, k=1, which='LM')
    spectral_radius = np.abs(eigenvalues).max()
    print(f"Spectral Radius: {spectral_radius}, alpha_max = {1/spectral_radius}")

    katz_centrality = calculate_katz(adj_matrix, alpha=alpha, beta=beta)
    index_mapping =  np.argsort(np.argsort(katz_centrality))

    mapped_row_indices = index_mapping[row_indices]
    mapped_col_indices = index_mapping[col_indices]

    adj_matrix = sp.csr_matrix((data, (mapped_row_indices, mapped_col_indices)), shape=(n, n), dtype=np.float64)
    new_katz = calculate_katz(adj_matrix, alpha=alpha, beta=beta)
    assert np.allclose(np.sort(new_katz), new_katz)

    return adj_matrix, np.sort(new_katz)


n = 100
p = 6/n
adj_matrix, katz_vector = generate_edge_list_and_katz_vector(n=n, p=p, alpha=0.01)
edges = get_edge_vector_from_adj_matrix(adj_matrix)


In [ ]:
import logging
h=logging.StreamHandler(); h.setFormatter(logging.Formatter(fmt="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")); r=logging.getLogger(); r.setLevel(logging.INFO); r.handlers.clear() if r.hasHandlers() else None; r.addHandler(h);


In [4]:
from greedy_algo_fast import greedy_search

w = np.exp(-9)
clusters_fast, _  = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w, max_interaction_dist=2)

Iteration 0 Objective: 0.030361978333363217 move_type split
Iteration 1 Objective: 0.022363023728179048 move_type split
Iteration 2 Objective: 0.01444448765126113 move_type split
Iteration 3 Objective: 0.013398665688751492 move_type split
Iteration 4 Objective: 0.012530081965618346 move_type split
Iteration 5 Objective: 0.011658279069397419 move_type merge-split
Iteration 6 Objective: 0.011376216807715209 move_type split
Iteration 7 Objective: 0.01135039377710733 move_type merge-split


In [5]:
n = 200
p = 6/n
adj_matrix, katz_vector = generate_edge_list_and_katz_vector(n=n, p=p, alpha=0.1)
edges = get_edge_vector_from_adj_matrix(adj_matrix)

Got Edge Indices
Spectral Radius: 5.942110909154865, alpha_max = 0.1682903626822792
Katz converged after 59 iterations.
Katz converged after 59 iterations.


In [9]:
from randcolorgraphs.algorithms.linear_scalarization.unoptimized_greedy_search import unoptimized_greedy_search_linear_scalarization
from OLD_greedy_algo_fast import greedy_search as old_greedy_search

def are_clusterings_equivalent(cluster1, cluster2):
    label_mapping = dict(zip(cluster1, cluster2))
    relabeled_cluster1 = np.array([label_mapping[label] for label in cluster1])
    return np.all(relabeled_cluster1 == cluster2)

w = np.exp(-9)

clusters_fast, obj_fast = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w)

print("-"*30)
#clusters, obj = old_greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w)
# Can go to n ~ 500; smaller w -> much slower
clusters, obj = unoptimized_greedy_search_linear_scalarization(katz_vector, adj_matrix.todense(), np.array([0]*len(katz_vector), dtype=np.int64), w=w, pam_cluster_dist=2)

assert are_clusterings_equivalent(clusters_fast, clusters)

Iteration 0 Objective: 25.18920030558933 move_type split
Iteration 1 Objective: 15.796090453770182 move_type split
Iteration 2 Objective: 6.763269429751096 move_type split
Iteration 3 Objective: 4.997437165230334 move_type split
Iteration 4 Objective: 3.7853425928302173 move_type split
Iteration 5 Objective: 2.6721151908469354 move_type split
Iteration 6 Objective: 1.792262913449909 move_type split
Iteration 7 Objective: 1.5036795746782023 move_type split
Iteration 8 Objective: 1.3140012240983416 move_type split
Iteration 9 Objective: 1.1321453970961055 move_type split
Iteration 10 Objective: 0.9536395707019998 move_type split
Iteration 11 Objective: 0.7820862585361328 move_type split
Iteration 12 Objective: 0.6342046248593394 move_type split
Iteration 13 Objective: 0.5363808762258084 move_type split
Iteration 14 Objective: 0.4431726731477142 move_type split
Iteration 15 Objective: 0.39196292724423015 move_type split
Iteration 16 Objective: 0.34535517633493085 move_type split
Iteration